In [ ]:
%autoreload 2

In [ ]:
import logging 
logging.basicConfig()
# log = logging.getLogger("blocking").setLevel("DEBUG")
# log = logging.getLogger("sql").setLevel("DEBUG")
# log = logging.getLogger("gammas").setLevel("DEBUG")
# log = logging.getLogger("expectation_step").setLevel("DEBUG")
# log = logging.getLogger("maximisation_step").setLevel("DEBUG")

In [ ]:
from pyspark.context import SparkContext, SparkConf
from pyspark.sql import SparkSession, Window
from pyspark.sql.types import StructType
import pyspark.sql.functions as f

# WARNING:
# These config options are appropriate only if you're running Spark locally!!!
conf=SparkConf()
conf.set('spark.driver.memory', '8g')
conf.set("spark.sql.shuffle.partitions", "8") 

sc = SparkContext.getOrCreate(conf=conf)

sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

In [ ]:
import sys 
sys.path.append("sparklink")
from sql import *
from blocking import *
from gammas import *
from params import *
from expectation_step import *
from maximisation_step import *
from iterate import *


In [ ]:
df = spark.read.parquet("data/fake_100.parquet")
df = df.dropDuplicates()
df.show()

In [ ]:
blocking_rules = []

blocking_rules.append('l.first_name = r.first_name  and  l.dob = r.dob')
blocking_rules.append('l.first_name = r.first_name  and  l.city = r.city')
blocking_rules.append('l.first_name = r.first_name  and  l.email = r.email')

blocking_rules.append('l.surname = r.surname  and  l.dob = r.dob')
blocking_rules.append('l.surname = r.surname  and  l.city = r.city')
blocking_rules.append('l.surname = r.surname  and  l.email = r.email')

blocking_rules.append('l.dob = r.dob  and  l.city = r.city')
blocking_rules.append('l.dob = r.dob  and  l.email = r.email')

blocking_rules.append('l.city = r.city  and  l.email = r.email')

In [ ]:
df_comparison = block_using_rules(df, blocking_rules, spark=spark)

In [ ]:

from gammas import *

gamma_settings = {
    "first_name": {
        "levels": 3,
        "case_expression": gammas_case_statement_3_levels("first_name", 0)
    },
    "surname": {
        "levels": 3
    },
    "dob": {
        "levels": 2
    },
    "city": {
        "levels": 2
    },
    "email": {
        "levels": 2
    }
}


# gamma_settings = {
#     "first_name": {
#         "levels": 3,
#         "case_expression": gammas_case_statement_3_levels("first_name", 0)
#     },
#     "surname": {
#         "levels": 2
#     }
    
# }

df_gammas = add_gammas(df_comparison, gamma_settings, spark, include_orig_cols = False)

In [ ]:
params = Params(gamma_settings)
df_e = iterate(df_gammas, spark, params, num_iterations=20)

In [ ]:
params.probability_distribution_chart()

In [ ]:
params.lambda_iteration_chart()

In [ ]:
params.pi_iteration_chart()